# Predicting Airline Delays in Python

The following is a demonstration of predicting potential flight delays using a publicly available airlines dataset. For this example, the dataset used is a small sample of what is more than two decades worth of flight data in order to ensure the download and import process would not take more than a minute or two.

## The Data

The data comes originally from [RITA](http://www.transtats.bts.gov/OT_Delay/OT_DelayCause1.asp) where it is described in detail. To use the entire 26 years worth of flight information to more accurately predict delays and cancellation please download one of the following and change the path to the data in the notebook: 

  * [2 Thousand Rows - 4.3MB](https://s3.amazonaws.com/h2o-airlines-unpacked/allyears2k.csv)
  * [5.8 Million Rows - 580MB](https://s3.amazonaws.com/h2o-airlines-unpacked/airlines_all.05p.csv)
  * [152 Million Rows (Years: 1987-2013) - 14.5GB](https://s3.amazonaws.com/h2o-airlines-unpacked/allyears.1987.2013.csv)

## Business Benefits

There are obvious benefits to predicting potential delays and logistic issues for a business. It helps the user make contingency plans and corrections to avoid undesirable outcomes. Recommendation engines can forewarn flyers of possible delays and rank flight options accordingly, other businesses might pay more for a flight to ensure certain shipments arrive on time, and airline carriers can use the information to better their flight plans. The goal is to have the machine take in all the possible factors that will affect a flight and return the probability of a flight being delayed.

### Load the H2O module and start an local H2O cluster

Connection to an H2O cloud is established through the `h2o.init` function from the `h2o` module. To connect to a pre-existing H2O cluster make sure to edit the H2O location with argument `myIP` and `myPort`.


In [26]:
import h2o
import os
import tabulate
import operator 
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
from h2o.estimators.deeplearning import H2ODeepLearningEstimator

In [27]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,1 hour 45 mins
H2O_cluster_timezone:,America/New_York
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.42.0.2
H2O_cluster_version_age:,5 days
H2O_cluster_name:,H2O_from_python_Ravichandran_y31k4h
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,13.53 Gb
H2O_cluster_total_cores:,16
H2O_cluster_allowed_cores:,16
H2O_cluster_status:,"locked, healthy"


### Import Data into H2O

We will use the `h2o.importFile` function to do a parallel read of the data into the H2O distributed key-value store. During import of the data, features Year, Month, DayOfWeek, and FlightNum were set to be parsed as enumerator or categorical rather than numeric columns. Once the data is in H2O, get an overview of the airlines dataset quickly by using `describe`.


In [28]:
airlines_hex = h2o.import_file(path = os.path.realpath("../data/allyears2k.csv"),
                               destination_frame = "airlines.hex")
airlines_hex.describe()

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


Rows:43978
Cols:31

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,IsArrDelayed,IsDepDelayed
type,int,int,int,int,int,int,int,int,enum,int,enum,int,int,int,int,int,enum,enum,int,int,int,int,enum,int,int,int,int,int,int,enum,enum
mins,1987.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,,1.0,,16.0,17.0,14.0,-63.0,-16.0,,,11.0,0.0,0.0,0.0,,0.0,0.0,0.0,0.0,0.0,0.0,,
mean,1997.5,1.4090909090909092,14.601073263904679,3.820614852880986,1345.8466613820756,1313.2228614307153,1504.6341303788884,1485.289167310928,,818.8429896766565,,124.81452913540424,125.0215626066189,114.31611109078271,9.317111936984317,10.007390655600112,,,730.1821905650503,5.381368059530624,14.168634184732058,0.024694165264450407,,0.0024785119832643593,4.047800291055638,0.2893764692712415,4.855031904175528,0.017015560282100075,7.620060450016793,,
maxs,2008.0,10.0,31.0,7.0,2400.0,2359.0,2400.0,2359.0,,3949.0,,475.0,437.0,402.0,475.0,473.0,,,3365.0,128.0,254.0,1.0,,1.0,369.0,201.0,323.0,14.0,373.0,,
sigma,6.344360901710614,1.8747113713439636,9.17579042586145,1.9050131191328963,465.3408991242338,476.25113999259963,484.347487903516,492.750434122701,,777.4043691636348,,73.97444166059017,73.40159463000927,69.63632951506104,29.840221962414844,26.438809042916446,,,578.4380082304243,4.201979939864827,9.905085747204327,0.15519314135784237,,0.049723487218862286,16.2057299044842,4.4167798987341245,18.61977622147568,0.40394018210151167,23.487565874106217,,
zeros,0,0,0,0,0,569,0,569,,0,,0,0,0,1514,6393,,,0,623,557,42892,,43869,7344,8840,7388,8914,7140,,
missing,0,0,0,0,1086,0,1195,0,0,0,32,1195,13,16649,1195,1086,0,0,35,16026,16024,0,9774,0,35045,35045,35045,35045,35045,0,0
0,1987.0,10.0,14.0,3.0,741.0,730.0,912.0,849.0,PS,1451.0,NA,91.0,79.0,nan,23.0,11.0,SAN,SFO,447.0,nan,nan,0.0,NA,0.0,nan,nan,nan,nan,nan,YES,YES
1,1987.0,10.0,15.0,4.0,729.0,730.0,903.0,849.0,PS,1451.0,NA,94.0,79.0,nan,14.0,-1.0,SAN,SFO,447.0,nan,nan,0.0,NA,0.0,nan,nan,nan,nan,nan,YES,NO
2,1987.0,10.0,17.0,6.0,741.0,730.0,918.0,849.0,PS,1451.0,NA,97.0,79.0,nan,29.0,11.0,SAN,SFO,447.0,nan,nan,0.0,NA,0.0,nan,nan,nan,nan,nan,YES,YES


### Building a GLM Model

Run a logistic regression model using function `h2o.glm` and selecting “binomial” for parameter `Family`. Add some regularization by setting alpha to 0.5 and lambda to 1e-05.

In [29]:
# Set predictor and response variables
myY = "IsDepDelayed"
myX = ["Dest", "Origin", "DayofMonth", "Year", "UniqueCarrier", "DayOfWeek", "Month", "Distance"]

# GLM - Predict Delays
glm_model = H2OGeneralizedLinearEstimator(
    family = "binomial",standardize = True, solver = "IRLSM",
    link = "logit", alpha = 0.5, model_id = "glm_model_from_python" )
glm_model.train(x               = myX,
               y               = myY,
               training_frame  = airlines_hex)


glm Model Build progress: |██████████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2OGeneralizedLinearEstimator : Generalized Linear Modeling
Model Key: glm_model_from_python


GLM Model: summary
    family    link    regularization                                 number_of_predictors_total    number_of_active_predictors    number_of_iterations    training_frame
--  --------  ------  ---------------------------------------------  ----------------------------  -----------------------------  ----------------------  ----------------
    binomial  logit   Elastic Net (alpha = 0.5, lambda = 1.507E-4 )  281                           175                            7                       airlines.hex

ModelMetricsBinomialGLM: glm
** Reported on train data. **

MSE: 0.21926317832437667
RMSE: 0.4682554626743576
LogLoss: 0.6284201306053259
AUC: 0.7019232382289031
AUCPR: 0.715565179443901
Gini: 0.40384647645780625
Null degrees of freedom: 43977
Residual degrees of freedom: 43802
Null deviance: 60855.95153848666
Residual deviance: 55273.32100752204
AIC: 55625.32100752204

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.3833979271338369
       NO    YES    Error    Rate
-----  ----  -----  -------  -----------------
NO     6207  14680  0.7028   (14680.0/20887.0)
YES    2429  20662  0.1052   (2429.0/23091.0)
Total  8636  35342  0.389    (17109.0/43978.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.383398     0.707203  283
max f2                       0.114321     0.846952  393
max f0point5                 0.547291     0.666724  183
max accuracy                 0.501047     0.649461  211
max precision                0.959763     1         0
max recall                   0.0862649    1         398
max specificity              0.959763     1         0
max absolute_mcc             0.547291     0.296624  183
max min_per_class_accuracy   0.527903     0.647049  194
max mean_per_class_accuracy  0.547291     0.648153  183
max tns                      0.959763     20887     0
max fns                      0.959763     23076     0
max fps                      0.0726623    20887     399
max tps                      0.0862649    23091     398
max tnr                      0.959763     1         0
max fnr                      0.959763     0.99935   0
max fpr                      0.0726623    1         399
max tpr                      0.0862649    1         398

Gains/Lift Table: Avg response rate: 52.51 %, avg score: 52.51 %
group    cumulative_data_fraction    lower_threshold    lift      cumulative_lift    response_rate    score     cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  --------  -----------------  ---------------  --------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.010005                    0.865505           1.83097   1.83097            0.961364         0.910409  0.961364                    0.910409            0.0183188       0.0183188                  83.0967   83.0967            0.0175049
2        0.0200555                   0.831088           1.62878   1.72964            0.855204         0.845166  0.908163                    0.877713            0.01637         0.0346888                  62.8779   72.9644            0.0308108
3        0.030015                    0.807682           1.57408   1.67803            0.826484         0.818951  0.881061                    0.858215            0.0156771       0.0503659                  57.4081   67.8025            0.0428493
4        0.0400655                   0.794741           1.52967   1.64081            0.803167         0.800673  0.861521                    0.84378             0.015374        0.0657399                  52.9674   

In [30]:
print("AUC of the training set : " + str(glm_model.auc()))
# Variable importances from each algorithm
# Calculate magnitude of normalized GLM coefficients
glm_varimp = glm_model.coef_norm()
for k,v in glm_varimp.items():
    glm_varimp[k] = abs(glm_varimp[k])
    
# Sort in descending order by magnitude
glm_sorted = sorted(glm_varimp.items(), key = operator.itemgetter(1), reverse = True)
table = tabulate.tabulate(glm_sorted, headers = ["Predictor", "Normalized Coefficient"], tablefmt = "orgtbl")
print("Variable Importances:\n\n" + table)

AUC of the training set : 0.7019232382289031
Variable Importances:

| Predictor        |   Normalized Coefficient |
|------------------+--------------------------|
| Origin.MDW       |               1.73309    |
| Origin.AUS       |               1.31225    |
| Origin.HNL       |               1.03038    |
| UniqueCarrier.WN |               0.974221   |
| Origin.ERI       |               0.961252   |
| Origin.ATL       |               0.897842   |
| Origin.TLH       |               0.895819   |
| Origin.MYR       |               0.871357   |
| Origin.BUR       |               0.866715   |
| Origin.LIH       |               0.866301   |
| Origin.PSP       |               0.845205   |
| UniqueCarrier.TW |               0.802473   |
| Origin.BNA       |               0.763528   |
| Dest.LYH         |               0.755978   |
| Origin.HPN       |               0.743651   |
| Origin.ABQ       |               0.739158   |
| Origin.PBI       |               0.718862   |
| Origin.STL       |

### Building a Deep Learning Model

Build a binary classfication model using function `h2o.deeplearning` and selecting “bernoulli” for parameter `Distribution`. Run 100 passes over the data by setting parameter `epoch` to 100.

In [31]:
# Deep Learning - Predict Delays
deeplearning_model = H2ODeepLearningEstimator(
    distribution = "bernoulli", model_id = "deeplearning_model_from_python",
    epochs = 100, hidden = [200,200],  
    seed = 6765686131094811000, variable_importances = True)
deeplearning_model.train(x               = myX,
                         y               = myY,
                         training_frame  = airlines_hex)

deeplearning Model Build progress: |█████████████████████████████████████████████| (done) 100%


Model Details
=============
H2ODeepLearningEstimator : Deep Learning
Model Key: deeplearning_model_from_python


Status of Neuron Layers: predicting IsDepDelayed, 2-class classification, bernoulli distribution, CrossEntropy loss, 97,602 weights/biases, 1.1 MB, 4,400,474 training samples, mini-batch size 1
    layer    units    type       dropout    l1    l2    mean_rate             rate_rms              momentum    mean_weight            weight_rms           mean_bias            bias_rms
--  -------  -------  ---------  ---------  ----  ----  --------------------  --------------------  ----------  ---------------------  -------------------  -------------------  -------------------
    1        284      Input      0.0
    2        200      Rectifier  0.0        0.0   0.0   0.07507370692978031   0.24492651224136353   0.0         -0.004405406823136837  0.18595975637435913  -0.9848622359400021  0.48988306522369385
    3        200      Rectifier  0.0        0.0   0.0   0.2748816372255187    0.2558143138885498    0.0         -0.05398843044258877   0.2675340175628662   -0.7228860164345484  1.1734318733215332
    4        2        Softmax               0.0   0.0   0.012427699869476783  0.011376690119504929  0.0         0.009954080560419243   0.5297868251800537   0.1415117050987444   0.31210005283355713

ModelMetricsBinomial: deeplearning
** Reported on train data. **

MSE: 0.12995973053289067
RMSE: 0.3604992795178524
LogLoss: 0.39649470459550645
Mean Per-Class Error: 0.19111141135960696
AUC: 0.8988111969097996
AUCPR: 0.9109915685548438
Gini: 0.7976223938195992

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.4046645554310208
       NO    YES    Error    Rate
-----  ----  -----  -------  ---------------
NO     3560  1141   0.2427   (1141.0/4701.0)
YES    738   4552   0.1395   (738.0/5290.0)
Total  4298  5693   0.1881   (1879.0/9991.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.404665     0.828917  233
max f2                       0.166623     0.894072  323
max f0point5                 0.621022     0.84011   156
max accuracy                 0.449933     0.813232  217
max precision                0.999896     1         0
max recall                   0.00416632   1         396
max specificity              0.999896     1         0
max absolute_mcc             0.449933     0.624872  217
max min_per_class_accuracy   0.477292     0.811531  208
max mean_per_class_accuracy  0.464839     0.812356  212
max tns                      0.999896     4701      0
max fns                      0.999896     4935      0
max fps                      0.000340982  4701      399
max tps                      0.00416632   5290      396
max tnr                      0.999896     1         0
max fnr                      0.999896     0.932892  0
max fpr                      0.000340982  1         399
max tpr                      0.00416632   1         396

Gains/Lift Table: Avg response rate: 52.95 %, avg score: 50.99 %
group    cumulative_data_fraction    lower_threshold    lift        cumulative_lift    response_rate    score      cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  ----------  -----------------  ---------------  ---------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.010009                    0.999999           1.88866     1.88866            1                1          1                           1                   0.0189036       0.0189036                  88.8658   88.8658            0.0189036
2        0.020018                    0.999949           1.88866     1.88866            1                0.999986   1                      

In [32]:
print("AUC of the training set : " + str(deeplearning_model.auc()))
deeplearning_model.varimp(table)

AUC of the training set : 0.8988111969097996


,variable,relative_importance,scaled_importance,percentage
0,DayofMonth,1.000000,1.000000,0.009892
1,UniqueCarrier.PS,0.989686,0.989686,0.009790
2,Year,0.978759,0.978759,0.009682
3,DayOfWeek,0.908262,0.908262,0.008985
4,UniqueCarrier.US,0.744716,0.744716,0.007367
...,...,...,...,...
279,Dest.LBB,0.181902,0.181902,0.001799
280,Dest.AMA,0.178747,0.178747,0.001768
281,Dest.missing(NA),0.000000,0.000000,0.000000
282,Origin.missing(NA),0.000000,0.000000,0.000000


### Shut down the cluster

Shut down the cluster now that we are done using it.

In [33]:
h2o.shutdown(prompt=False)

H2O session _sid_a81b closed.


C:\Users\Ravichandran\AppData\Local\Temp\ipykernel_34948\1954269801.py:1: H2ODeprecationWarning: Deprecated, use ``h2o.cluster().shutdown()``.
  h2o.shutdown(prompt=False)
